In [1]:
#Import required libraries
import gspread
import sys
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
#Import shared functions
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

In [5]:
sql = 'select * from crowdsdb.dbo.vw_consolidated_socialdistancing'

In [6]:
consolidated_sql = (pd.read_sql(con = engine, sql = sql)
                    .fillna(value = np.nan, axis = 1))

In [8]:
consolidated_sql.tail()

,source_survey,encounter_datetime,site_id,location_adddesc,city_agency,encounter_type,amenity,patroncount,borough,precinct,patrol_boro
27388,DPR Crowds,2020-03-25 10:54:28,R005-ZN01,NaN,DPR,NaN,NaN,10.0,Staten Island,NaN,NaN
27389,DPR Crowds,2020-03-25 10:49:53,R005,NaN,DPR,NaN,NaN,6.0,Staten Island,NaN,NaN
27390,DPR Crowds,2020-03-25 10:49:03,R045-01,NaN,DPR,NaN,NaN,8.0,Staten Island,NaN,NaN
27391,DPR Crowds,2020-03-25 10:44:20,R030-99,NaN,DPR,NaN,NaN,18.0,Staten Island,NaN,NaN
27392,DPR Crowds,2020-03-25 08:07:40,M108T01,NaN,DPR,NaN,NaN,17.0,Manhattan,NaN,NaN
